                    Implementing a simplified version of Locality sensitive hashing

In [1]:
# Sample Sentences
a = "The quick brown fox jumped over the little lazy dog"
b = "An elephant is jumping around in the streets"
c = "You have to be quick, being lazy will take you nowhere"
d = "The little lazy dog jumped over the quick brown fox"

We will be make k shingles with value of k being = 3. For larger datasets with large texts, we have to take the size of k more so that only those texts which are similar get closer vectors.

In [2]:
k = 3
def shingle(text: str, k = 3):
    shingle_list = []
    for i in range(0,len(text) - k):
        shingle_list.append(text[i:i+k])
    return list(set(shingle_list))

In [3]:
# Demonstration of formation of shingles
A = shingle(a,k)
print(A)

['uic', ' fo', 'lit', ' br', 'ped', 'le ', 'ox ', 'ove', 'own', ' li', ' la', 'azy', 'mpe', 'e l', 'he ', ' ju', 'zy ', 'fox', 'The', 'tle', ' do', 'y d', 'r t', 'ick', 'ck ', 'k b', 'n f', 'wn ', 'ump', ' qu', 'bro', 'x j', ' ov', 'ver', ' th', 'itt', 'ttl', 'ed ', 'laz', 'row', 'e q', 'jum', 'd o', 'the', 'qui', 'er ']


In [4]:
import random
random.seed(42)

def vocab(lest : list, k: int):
    newlist = []
    for texts in lest:
        shin = shingle(texts, k)
        for shingl in shin:
            newlist.append(shingl)
    thelist = list(set(newlist))
    random.shuffle(thelist) 
    return thelist


In [5]:
texts = [a,b,c,d]
vocab = vocab(texts, k)
print(vocab)
print("length of vocab is: ", len(vocab))

['be ', ' yo', ' in', 'dog', 'o b', ' br', ' ar', 'e t', 'own', 'ele', 'An ', 'nt ', 'qui', 'le ', 'wn ', ' is', 'tak', 'k, ', 'ke ', 'e s', 'whe', ' ha', 'og ', 'is ', 'wil', 'g l', 'k b', 'fox', ', b', 'ump', 'ake', 'ped', 't i', 'ed ', ' wi', 'oun', ' fo', 'her', 'hav', 'y d', 'er ', 'ou ', 'the', 've ', 'eet', 'now', 'ox ', 'pin', 'r t', ' st', 'itt', 'tre', 'zy ', 'he ', 'e q', 'ng ', ' ov', 'n e', 'e l', 'ick', 'ove', 'bei', 'The', 'aro', 'ver', 'nd ', 'u n', ' do', 'll ', 'mpe', 'd o', ' ju', ' li', 'g a', 'bro', 'ck ', 'n t', ' ta', 'x j', 'ein', 'ill', 'ant', ' el', 'n f', 's j', 'laz', 'han', 'mpi', 'to ', 'jum', 'in ', ' be', 'pha', 'str', 'eph', 'uic', 'lep', ' no', 'ave', 'ck,', 'owh', 'lit', 'd i', ' to', 'y w', ' la', 'row', 'tle', 'rou', 'e y', ' th', 'g j', 'ree', 'ing', ' qu', 'und', 'l t', 'azy', 'u h', 'you', 'You', 'ttl']
length of vocab is:  122


                            Creating One hot vectors for all the text sentences.

In [6]:
one_hot_vectors = []
for text in texts:
    a_1hot = [1 if x in text else 0 for x in vocab]
    one_hot_vectors.append(a_1hot)

print(len(one_hot_vectors))
print(len(one_hot_vectors[0]))

4
122


Here we have 4 122 dimensioned vectors which are one hot encoded for the k shingles. We will now use minhashing to get the different types of values. 

In [7]:
# MINHASHING to get signature
import numpy as np

def findones(arr, lest):
    for i in arr:
        if(lest[i] == 1):
            return int(i)

def get_signature(lest: list, n = 20):
    length = len(lest[0])
    random_matrix = []
    for i in range(n):
        random_matrix.append(np.random.randint(0,length,length))
    signatures = []
    for les in lest:
        sig = np.ones(n)
        for i in range(n):
            sig[i] = findones(random_matrix[i], les)
        signatures.append(sig)
    return signatures

    

In [8]:
g = get_signature(one_hot_vectors)
print(g)

[array([ 89.,  33.,  12.,   5.,  46.,  70.,  78.,  12.,  70.,  33.,  83.,
        14.,  31.,  72., 117.,  58., 117.,  85.,  46.,  95.]), array([ 89.,  35.,  55.,  11.,   6.,  93.,  71.,  32.,  44.,   9.,  65.,
        42.,   6.,  63.,  19.,  49., 112.,  93.,  96., 115.]), array([ 91., 100.,  12.,  21.,  77.,  20., 103.,  12., 116.,   4.,  79.,
       100.,   4.,  55., 117.,  12.,  41.,  85.,  88., 116.]), array([ 89.,  33.,  12.,   5.,  40.,  70.,  71.,  12.,  70.,  33.,  83.,
        14.,  22.,  72., 117.,  58., 111., 111.,  40.,  95.])]


In [9]:
# Jaccard function to compare two vectors
def jaccard(a, b):
    ans = 0
    for i in range(len(a)):
        if a[i] in b:
            ans = ans +1
    return ans/len(a)

In [10]:
# Comparing a and b
print(jaccard(shingle(a), shingle(b)), jaccard(g[0], g[1]))
# Comparing a and c
print(jaccard(shingle(a), shingle(c)), jaccard(g[0], g[2]))
# Comparing a and d
print(jaccard(shingle(a), shingle(d)), jaccard(g[0], g[3]))

0.13043478260869565 0.05
0.1956521739130435 0.25
0.9347826086956522 0.75


We have shown that texts which match in their semantics also match in their hashing, so we will use these signatures and Perform an LSH (Locality Sensitiv Hashing on them)

We will make bands of the subvectors and will make candidate pairs if the hasing of the subvecotrs falls into the same bucket

In [11]:
def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

In [12]:
bands = 10
bands_a = split_vector(g[0], bands)
bands_b = split_vector(g[1], bands)
bands_c = split_vector(g[2], bands)
bands_d = split_vector(g[3], bands)

In [13]:
print(bands_a)
print(bands_b)

[array([89., 33.]), array([12.,  5.]), array([46., 70.]), array([78., 12.]), array([70., 33.]), array([83., 14.]), array([31., 72.]), array([117.,  58.]), array([117.,  85.]), array([46., 95.])]
[array([89., 35.]), array([55., 11.]), array([ 6., 93.]), array([71., 32.]), array([44.,  9.]), array([65., 42.]), array([ 6., 63.]), array([19., 49.]), array([112.,  93.]), array([ 96., 115.])]


In [14]:
# def check_bands(banda, bandb):
#     for i in range(len(banda)):
#         equal = 1
#         for j in range(len(banda[i])):
#             if banda[i] != bandb[i]:
#                 equal = 0
#                 break;

#         if banda[i].all() == bandb[i].all():
#             print(banda[i].all())
#             print(bandb[i].all())
#             print("Candidate pairs ", banda[i])
#             return
#     print("Not Candidate pairs")

def check_bands(banda, bandb):
    for i in range(len(banda)):
        if list(banda[i]) == list(bandb[i]):
            print("Candidate pairs ", banda[i])
            return
    print("Not Candidate pairs")
    return

In [15]:
# Checking the candidate pairs of statements
check_bands(bands_a, bands_b)
check_bands(bands_a, bands_c)
check_bands(bands_a, bands_d)
check_bands(bands_b, bands_d)


Not Candidate pairs
Not Candidate pairs
Candidate pairs  [89. 33.]
Not Candidate pairs


Hense, we can see sentence a = "The quick brown fox jumped over the little lazy dog" and sentence 
d = "The little lazy dog jumped over the quick brown fox" are a candidate pair